In [57]:
import pandas as pd
import numpy as np
import tpqoa

In [58]:
class ConTrader(tpqoa.tpqoa):

    def __init__(self, config_file, instrument, bar_length, window, units):
        super().__init__(config_file)
        self.position = 0
        self.instrument = instrument
        self.window = window
        self.bar_length = bar_length
        self.units = units
        self.tick_data = pd.DataFrame()
        self.min_length = self.window+1
        self.profits = [] # list with every PL

# utilities
    def prepare_data(self):
        self.data["returns"] = np.log(self.data.mid.div(self.data.mid.shift(periods=1)))
        self.data["position"] = -np.sign(self.data.returns.rolling(window=self.window).mean())

    def resample_data(self):
        self.data = self.tick_data.resample(self.bar_length, label="right").last().ffill().iloc[:-1]

    def report_trade(self, order, going):
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cum_pl = sum(self.profits)

        print("\n"+60*"-")
        print("{} | {}".format(time, going))
        print("{} | units: {} | price: {} | P&L: {} | Cum P&L: {}".format(time, units, price, pl, cum_pl))
        print(60*"-")

    
# trading
    def on_success(self, time, bid, ask):
        print(self.ticks, end=" ")

    # collect and resample tick data
        df = pd.DataFrame({"bid": bid, "ask": ask, "mid": (bid+ask)/2}, index=[pd.to_datetime(time)])
        self.tick_data = self.tick_data.append(df)
        self.resample_data()

    # prepare data & define strategy
        self.prepare_data()

    # live trading
        if len(self.data) > self.min_length - 1:
            self.min_length += 1
            if self.data.position.iloc[-1] == 1:
                if self.position == 0:
                    order = self.create_order(instrument=self.instrument, units=self.units, suppress=True, ret=True)
                    self.report_trade(order, "Going Long")
                elif self.position == -1:
                    order = self.create_order(instrument=self.instrument, units=self.units*2, suppress=True, ret=True)
                    self.report_trade(order, "Closing Prev Position & Going Long")
                self.position = 1
            elif self.data.position.iloc[-1] == -1:
                if self.position == 0:
                    order = self.create_order(instrument=self.instrument, units=-self.units, suppress=True, ret=True)
                    self.report_trade(order, "Going Short")
                elif self.position == 1:
                    order = self.create_order(instrument=self.instrument, units=-self.units*2, suppress=True, ret=True)
                    self.report_trade(order, "Closing Prev Position & Going Short")
                self.position = -1

In [59]:
trader = ConTrader("oanda.cfg", "EUR_USD", "5s", 1, 100000)

In [60]:
trader.stream_data(instrument=trader.instrument, stop=30)

1 

KeyboardInterrupt: 

In [48]:
trader.data

,bid,ask,mid,returns,position
2020-12-04 18:22:10+00:00,1.21464,1.21470,1.214670,NaN,NaN
2020-12-04 18:22:15+00:00,1.21459,1.21465,1.214620,-0.000041,1.0
2020-12-04 18:22:20+00:00,1.21459,1.21467,1.214630,0.000008,-1.0
2020-12-04 18:22:25+00:00,1.21459,1.21467,1.214630,0.000000,-1.0
2020-12-04 18:22:30+00:00,1.21460,1.21466,1.214630,0.000000,-1.0
2020-12-04 18:22:35+00:00,1.21460,1.21466,1.214630,0.000000,-1.0
2020-12-04 18:22:40+00:00,1.21462,1.21468,1.214650,0.000016,-1.0
2020-12-04 18:22:45+00:00,1.21460,1.21466,1.214630,-0.000016,1.0
2020-12-04 18:22:50+00:00,1.21467,1.21473,1.214700,0.000058,-1.0
2020-12-04 18:22:55+00:00,1.21470,1.21477,1.214735,0.000029,-1.0


In [54]:
trader.stream_data(instrument=trader.instrument, stop=70)
close_positions = trader.create_order(instrument=trader.instrument, units=-trader.position*trader.units, suppress=True, ret=True)
trader.report_trade(close_positions, "Closing All Positions")

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 
------------------------------------------------------------
2020-12-04T18:30:26.087571321Z | Going Short
2020-12-04T18:30:26.087571321Z | units: -100000.0 | price: 1.215 | P&L: 24.0 | Cum P&L: 24.0
------------------------------------------------------------
17 18 19 20 21 22 23 24 25 26 27 28 
------------------------------------------------------------
2020-12-04T18:30:46.533067737Z | Closing Prev Position & Going Long
2020-12-04T18:30:46.533067737Z | units: 200000.0 | price: 1.21507 | P&L: 0.0 | Cum P&L: 24.0
------------------------------------------------------------
29 30 31 32 33 
------------------------------------------------------------
2020-12-04T18:30:50.673637912Z | Closing Prev Position & Going Short
2020-12-04T18:30:50.673637912Z | units: -200000.0 | price: 1.21504 | P&L: 25.0 | Cum P&L: 49.0
------------------------------------------------------------
34 35 36 37 
------------------------------------------------------------
2020

In [56]:
trader.position

-1